In [1]:
from datetime import datetime as dt
import requests
import glob
import json
from bs4 import BeautifulSoup
import pandas as pd
import spacy
import urllib3
import time
import traceback
import os
from dotenv import load_dotenv
from itertools import chain
import ast
# df = pd.read_pickle('Data_Words/Data/job_words_indeed.pkl')
# description_sentences = df['description_sentences']
now = int(time.time() * 1000000)
load_dotenv('Data_Words/.env')
scraper_api = os.getenv("API_KEY")
from pathlib import Path
import numpy as np
from spacy.matcher import Matcher
cwd = Path.cwd()
nlp = spacy.load("en_core_web_lg")
stopwords = list(nlp.Defaults.stop_words)

#### Names and Types


In [2]:
def url_next(keyword, location, days, last_id=None, page_count=0):
    """Create start url for ids if lastId = None.  Otherwise uses id from previous
    request"""
    url = f"https://www.indeed.com/jobs?q={keyword}&l={location}&radius=15&sort=date&fromage={days}"
    if page_count == 0:
        return url
    else:
        return f"{url}&start={page_count * 10}&vjk={last_id}"


# u = url_next('data', 'San+Diego+CA', 14, last_id=None, page_count=0)
# 'https://www.indeed.com/jobs?q=data&l=San+Diego+CA&radius=15&sort=date&fromage=14'


def file_name_ids(folder_path, keyword, location):
    """creates json file name for ids using keyword and location"""
    file_name = f"{folder_path}/indeedIds_{keyword}_{location}.json"
    return file_name

#### Storing and Retrieving


In [3]:
def dd(any_list):
    """Converts list to set then back to list for the purpose of removing duplicates"""
    list_to_set = set(any_list)
    return list(list_to_set)


def j_load(file_name):
    """Retreives existing ids or creates new file.  Returns tuple with id list and count.
    Dedupes as precaution."""
    try:
        with open(file_name, "r", encoding="utf-8") as f:
            ja = json.load(f)
        og_ids = ja[file_name]
        og_list_dd = dd(og_ids)
        print(f"Starting with {len(og_list_dd)} existing ids for {file_name}")
        return (len(og_list_dd), og_list_dd)
    except (FileNotFoundError, TypeError, KeyError, ValueError):
        print(f"No file found using {file_name} starting fresh.")
        return (0, [])


def j_dump(file_name, ids):
    """saves ids as json"""
    with open(file_name, "w", encoding="utf-8") as f:
        json.dump({file_name: dd(ids)}, f)
    print(f"{len(ids)} ids saved to {file_name}")

#### Scrape


In [4]:
# Uses proxies and redirects to automatically scrape id html.  Requires key.
def scrape(retries, api_key, url):
    """request and return html from job listing page with retries
    and results printed.  Single thread.  Returned html is used to
    make next url.  Returns r"""
    tries = 0
    while tries <= retries:
        scraper_api = api_key
        payload = {"api_key": scraper_api, "url": url}
        print(f"Calling: {url}")
        r = requests.get("http://api.scraperapi.com", params=payload, timeout=30)
        try:
            if r.headers["sa-statusCode"] == "200":
                print(
                    f"{r.headers['Date']}   {r.headers['sa-final-url']} \
                      status: {r.headers['sa-statusCode']}"
                )
            return r
        except KeyError:
            print(f"{r.headers}")
            tries += 1
        print(rf"Try {tries} of {retries} n\ {r.headers}")
    return print(f"{url} has problems")


def calling(keyword, location, days, api_key, last_id, page_count):
    """Bundles methods and returns id html."""
    id_url = url_next(keyword, location, days, last_id, page_count)
    r = scrape(3, api_key, id_url)
    return r

# scrape(retries=3, api_key=scraper_api, url='https://www.indeed.com/jobs?q=data&l=Tampa&radius=15&sort=date&fromage=7')


#### Get Ids


In [5]:
def get_ids(r):
    """convert search page response to list of ids."""
    soup = BeautifulSoup(r.text, "html.parser")
    td_soup = soup.find_all("h2")
    jk_list = []
    for soup in td_soup:
        aa = soup.a
        try:
            jk = aa.attrs["data-jk"]
            jk_list.append(jk)
        except AttributeError:
            pass
    return jk_list


def get_new_ids(r, file_name, old_ids):
    # takes allIds and pageCount from previous loop if available
    """Returns a uniques set of ids consisting of old ids and this loop."""
    # loads saved file if first loop

    if len(old_ids) == 0:
        existing_ids_load = j_load(file_name)
        existing_id_count = existing_ids_load[0]
        existing_ids = existing_ids_load[1]
        existing_id_dd = dd(existing_ids)
        print(f"starting id count is {existing_ids_load[0]}")
    else:
        existing_id_dd = dd(old_ids)
        existing_id_count = len(existing_id_dd)
    try:
        loop_id_list = get_ids(r)
        all_unique = dd(existing_id_dd + loop_id_list)
    except (NameError, TypeError, AttributeError):
        all_unique = []
    all_unique_count = len(all_unique)
    loop_unique_count = all_unique_count - existing_id_count
    print(
        f"\n{dt.now()} page_unique_count: {loop_unique_count} currentCount: {all_unique_count}"
    )
    return all_unique


def zero_count(loop_count):
    """Returns false when 0 new ids returned twice in a row.
    Used to end scraping"""
    if len(loop_count) <= 2:
        return True
    if loop_count[-1] + loop_count[-2] > 0:
        return True
    return False


def just_ids(keyword=None, location=None, api_key=None, days=None, folder_path=None):
    """Assembles methods for IDs.  Save and count all.  Keyboard interupt causes end and save"""
    current_id_list = dd([])
    loop_count_list = []
    page_count = 0
    file_name = file_name_ids(folder_path, keyword, location)
    # breaks loop when 2 consecutive loops return 0 new ids
    try:
        while zero_count(loop_count_list):
            try:
                last_id = current_id_list[-1]
            except IndexError:
                last_id = str()
            try:
                r = calling(keyword, location, days, api_key, last_id, page_count)
            except urllib3.exceptions.ReadTimeoutError as e:
                print(f"{e}")
                j_dump(file_name, current_id_list)
            newest_ids = get_new_ids(r, file_name, current_id_list)
            try:
                loop_count = len(newest_ids) - len(current_id_list)
            except TypeError:
                loop_count = len(newest_ids)
            loop_count_list.append(loop_count)
            page_count += 1
            for new in newest_ids:
                current_id_list.append(new)
    except KeyboardInterrupt:
        pass
    j_dump(file_name, dd(current_id_list))
    print(
        f"Finished!  New id count is {len(dd(current_id_list))} \
        Saved to: {file_name}"
    )

In [6]:
# ids_san_diego = just_ids(
#     keyword="data",
#     location="San+Diego+CA",
#     api_key=scraper_api,
#     days=7,
#     folder_path="Data_Words/Indeed_ids",
# )

#### Filter Ids for use


In [7]:
def get_df_ids(
    file="Data_Words/Data/job_words_indeed.pkl",
):
    """Retrieves main df from csv and returns the Job Id column as a list"""
    """Used later to filter out ids that have already been processed"""
    try:
        df = pd.read_pickle(file)
        df_ids = list(df["job_id"])
        return df_ids
    except FileNotFoundError:
        print(f"no pkl found in {file}")
        return ["x"]

In [8]:
def get_job_dict(id_tuple):
    """Makes api call and converts html response to soup, extracts specific json, converts to dict return"""
    keyword = id_tuple[0]
    location = id_tuple[1]
    id = id_tuple[2]
    url = id_tuple[3]
    r = scrape(3, scraper_api, url).text
    try:
        # create soup from r
        soup = BeautifulSoup(r, "html.parser")
        # find and extract json from soup
        j_soup = soup.find("script", type="application/ld+json").text
        # convert json to dict
        d_soup = json.loads(j_soup)
        # Append data from tuple
        d_soup.update(
            {
                "jobId": id,
                "jobKeyword": keyword,
                "jobSearchLocation": location,
            }
        )
        # print(f'{dt.now()}get_job_dict try')
        return d_soup
    except AttributeError:
        # print(f'{dt.now()}get_job_dict ex')
        return {"jobId": id, "jobKeyword": keyword, "jobSearchLocation": location}


# test_get_job_dict = get_job_dict(test_tup)
# test_get_job_dict

In [9]:
# Ids ready
def ids_warming(path):
    """Extract ids from json files.  Ouput list of tuples with keyword, location, id, URL"""
    id_tuple_list = []
    # get ids that have already run
    df_ids = get_df_ids()
    # list of id file names
    ids = glob.glob(f"{path}/indeedIds_*.json")
    # loop through file names extract kw and location
    for id in ids:
        id_path = id.split("/")[-1]
        id_name = id_path.split("_")
        kw = id_name[1]
        loc = id_name[2].split(".")[0]
        # open each id file
        with open(id, "r", encoding="utf-8") as f:
            j_ids = json.load(f)[id]
            # if id already in df skip
            for j in j_ids:
                if j in df_ids:
                    pass
                elif kw in ("test", "write"):
                    pass
                # not in df... create tup using (kw,loc,id) and append to list
                else:
                    url = f"https://www.indeed.com/m/viewjob?jk={j}"
                    id_tup = (kw, loc, j, url)
                    id_tuple_list.append(id_tup)
    # return set of all unique unprocessed tuples
    tuple_dd_list = dd(id_tuple_list)
    print(f"{len(tuple_dd_list)} unique job ids will be processed")
    return tuple_dd_list


In [10]:
# This func does not work as stand alone
def li_li_li_list(description):
    """Takes description dict and returns li values as list"""
    # Resoup dict
    d_soup = BeautifulSoup(description, "html.parser")
    # get list of li tag text
    lis = d_soup.find_all("li")
    tagList = []
    for l in lis:
        # get just li text and put back in list
        bullet = l.text
        tagList.append(bullet)
    # print(f'{dt.now()}li_li_li_list')
    return tagList

In [11]:
# test_sen = test_check['description_list']

def spacy_proper(doc):
   """Takes string as input and returns a list of Proper Nouns"""
   pn_list = []
   for tok in doc:
      if tok.pos_ == 'PROPN':
       pn_list.append(tok.text)
      else:
        pass
   return pn_list

def sentence_parse_proper(sentences):
   """Parses list of sentences and returns list of proper nouns"""
   col_lists = []
   try:
      for sentence in sentences:
           ss= sentence.strip()
           doc = nlp(ss)
           pn = spacy_proper(doc)
           col_lists.append(pn)
   except ValueError:
      col_lists.append([])
   return set(chain.from_iterable(col_lists))


In [12]:
def pattern_lower(csv_file, column_name):
    """Formats column from csv file into patterns for matching"""
    pattern_list = []
    df = pd.read_csv(csv_file)
    word_list = list(df[column_name])
    clean_word_list = [str(t).lower().strip() for t in word_list if t is not np.nan]
    split_list = [t.split() for t in clean_word_list ]
    for i in range(len(split_list)):
        words = []
        sentence = split_list[i]
        for w in sentence:
            pattern = dict(LOWER = str(w))
            words.append(pattern)
        pattern_list.append(words)
    # print(f'{column_name} now contains {len(pattern_list)} keywords')
    return pattern_list


def data_word_match(sentence, csv_file, column_name):
    matcher=Matcher(nlp.vocab)
    word_patterns = pattern_lower(csv_file, column_name)
    matcher.add(column_name, word_patterns, greedy='FIRST')
    doc = nlp(sentence)
    matches = matcher(doc)
    words = []
    for match_id, start, end in matches:
        span = doc[start:end]
        words.append(span.text)
    return list(words)

## TODO use lemmatization to match base words instead of exact

In [13]:
def sentence_parse_data_words (sentences, csv_file, column_name):
   """Takes string as input and returns a list of Proper Nouns"""
   words_lists = []
   try:
      for sentence in sentences:
           words = data_word_match(sentence,csv_file, column_name)
           words_lists.append(words)
   except ValueError:
      words_lists.append([])
   return set(chain.from_iterable(words_lists))

#### Extract and Catch

def check_and_extract(full_dict):
    """This function is designed to catch errors in the dictionary.
    Function li_li_li_list extracts list of values.  Dict with data and lables returned
    """
    try:
        company = full_dict["hiringOrganization"]["name"]
    except KeyError:
        company = "Unavailable"
    except TypeError:
        company = "Unavailable"
    try:
        title = full_dict["title"]
    except TypeError:
        title = "Unavailable"
    except KeyError:
        title = "Unavailable"
    try:
        base_salary_low = full_dict["baseSalary"]["value"]["minValue"]
    except KeyError:
        base_salary_low = "Unavailable"
    except TypeError:
        base_salary_low = "Unavailable"
    try:
        base_salary_high = full_dict["baseSalary"]["value"]["maxValue"]
    except KeyError:
        base_salary_high = "Unavailable"
    except TypeError:
        base_salary_high = "Unavailable"
    try:
        salary_period = full_dict["baseSalary"]["value"]["unitText"]
    except KeyError:
        salary_period = "Unavailable"
    except TypeError:
        salary_period = "Unavailable"
    try:
        date_posted = full_dict["datePosted"]
    except KeyError:
        date_posted = "Unavailable"
    except TypeError:
        date_posted = "Unavailable"
    try:
        date_expires=full_dict["date_expires"]
    except TypeError:
        date_expires = "Unavailable"
    except KeyError:
        date_expires = "Unavailable"
    try:
        employment_type = full_dict["employmentType"]
    except KeyError:
        employment_type = "Unavailable"
    except TypeError:
        employment_type = "Unavailable"
    try:
        location = full_dict["jobLocation"]["address"]["addressLocality"]
    except KeyError:
        location = "Unavailable"
    except TypeError:
        location = "Unavailable"
    try:
        description_raw = full_dict["description"]
    except KeyError:
        description_raw = "Unavailable"
    except TypeError:
        description_raw = "Unavailable"
    try:
        description_list = li_li_li_list(description_raw)
    except KeyError:
        description_list = "Unavailable"
    except TypeError:
        description_list = "Unavailable"
    skills = sentence_parse_data_words (description_list, 'Data_Words/Data/snow_words.csv', 'Data_Skills')
    data_skills = [x.lower() for x in skills]
    technology = sentence_parse_data_words (description_list, 'Data_Words/Data/snow_words.csv', 'Data_Technology')
    data_technology = [x.lower() for x in technology]
    propers= sentence_parse_proper(description_list)
    proper_nouns = [x.lower() for x in propers if x.lower() not in data_skills and x.lower() not in data_technology]
    job_id = full_dict["jobId"]
    keyword = full_dict["jobKeyword"]
    search_location = full_dict["jobSearchLocation"]
    print(
        f"{job_id}  {keyword}   {search_location}  {company}  {title}  {date_posted}\
          {employment_type}  {location}  \n  {proper_nouns} \n {data_skills} \n {data_technology}"
    )
    job_dict = {
        "job_id": job_id,
        "entered": time.strftime("%D %T"),
        "search_keyword": keyword,
        "search_location": search_location,
        "job_company": company,
        "job_title": title,
        "job_date_posted": date_posted,
        "job_date_expires": date_expires,
        "pay_low": base_salary_low,
        "pay_high": base_salary_high,
        "pay_period": salary_period,
        "job_type": employment_type,
        "job_location": location,
        "description_sentences": description_list,
        "proper_nouns": proper_nouns,
        "data_skills": data_skills,
        "data_technology": data_technology
    }
    return job_dict


# test_check = check_and_extract(test_get_job_dict)
# test_check


#### Get and Clean up words


In [14]:
def come_together():
    """runs scraped ids and returns list df"""
    dict_list = []
    tup_list = ids_warming(path="Data_Words/Data")  # returns list of id tuples  ->list
    er_count = 0
    try:
        for tup in tup_list:
            try:
                job_dict = get_job_dict(
                    tup
                )  # takes one id tuple and outputs dict with data -> dict
                new_job_dict = check_and_extract(
                    job_dict
                )  # Takes description key from job dict and converts to list (description_list:[]) -> dict
                dict_list.append(new_job_dict)
                print(f"{len(dict_list)} jobs out of {len(tup_list)} processed")
            except TimeoutError:
                traceback.print_exc()
                er_count += 1
                er_retry = er_count * 60
                print(f"Error has occured.  Will retry in {er_retry}")
        df_new = pd.DataFrame(dict_list)
        df_main = pd.read_pickle("Data_Words/Data/job_words_indeed.pkl")
        combo_df = pd.concat([df_main, df_new], ignore_index=True)
        combo_df.to_pickle(f"Data_Words/Data/job_words_indeed.pkl")
        combo_df.to_pickle(f"Data_Words/BackUps/job_words_indeed{now}.pkl")
        return combo_df
    except KeyboardInterrupt:
        df_new = pd.DataFrame(dict_list)
        df_main = pd.read_pickle("Data_Words/Data/job_words_indeed.pkl")
        combo_df = pd.concat([df_main, df_new], ignore_index=True)
        combo_df.to_pickle(f"Data_Words/Data/job_words_indeed.pkl")
        combo_df.to_pickle(f"Data_Words/BackUps/job_words_indeed{now}.pkl")
        return df_new

In [15]:
ids_remote = just_ids(keyword='snowflake', location='Remote', api_key=scraper_api, days=14,
             folder_path='Data_Words/Data')
ids_san_diego = just_ids(
    keyword="snowflake",
    location="San+Diego+CA",
    api_key=scraper_api,
    days=14,
    folder_path="Data_Words/Data",
)
ids_tampa = just_ids(keyword='snowflake', location='Tampa', api_key=scraper_api, days=14,
             folder_path='Data_Words/Data')

df = come_together()


Calling: https://www.indeed.com/jobs?q=snowflake&l=Remote&radius=15&sort=date&fromage=14
Mon, 29 Apr 2024 02:18:05 GMT   https://www.indeed.com/jobs?q=snowflake&l=Remote&radius=15&sort=date&fromage=14                       status: 200
No file found using Data_Words/Data/indeedIds_snowflake_Remote.json starting fresh.
starting id count is 0

2024-04-28 19:18:05.503931 page_unique_count: 15 currentCount: 15
Calling: https://www.indeed.com/jobs?q=snowflake&l=Remote&radius=15&sort=date&fromage=14&start=10&vjk=c56eed9ef92308af
Mon, 29 Apr 2024 02:18:06 GMT   https://www.indeed.com/jobs?q=snowflake&l=Remote&radius=15&sort=date&fromage=14&start=10&vjk=c56eed9ef92308af                       status: 200

2024-04-28 19:18:07.326629 page_unique_count: 15 currentCount: 30
Calling: https://www.indeed.com/jobs?q=snowflake&l=Remote&radius=15&sort=date&fromage=14&start=20&vjk=7a970c9b83ecaa70
Mon, 29 Apr 2024 02:18:08 GMT   https://www.indeed.com/jobs?q=snowflake&l=Remote&radius=15&sort=date&fromage=1

In [17]:
from snot import snowy

connection_parameters = {
    "user": "Kona",
    "password": "ngFWnMT9cuG6XM6",
    "account": "bepcfpi-gf10139",
    "role": "ACCOUNTADMIN",
    "warehouse": "LOADER",
    "database": "PC_DBT_DB",
    "schema": "RAW",
}



snowy.df_to_table(df, "job_words_raw", connection_parameters)


Success!  280 rows added to job_words_raw in 1 chunks


In [1]:
df

NameError: name 'df' is not defined